# 📏 Calibration for v11 (LIDAR-based Grasp)

ใช้สำหรับ calibrate ค่าต่างๆ ก่อนใช้งาน `11_lidar_grasp_v11.ipynb`

## ค่าที่ต้อง Calibrate:
1. **LIDAR_TO_GRIPPER_OFFSET** - ระยะจาก LIDAR ถึงปลาย gripper
2. **Z_MEASURE** - ความสูงที่ใช้วัด LIDAR
3. **PIXELS_PER_MM** - สเกลกล้อง (ถ้ายังไม่มี)

## 🔌 Wiring TF-Luna + ESP32

```
  ┌─────────────────────────────────────────────────────┐
  │                    ESP32 DevKit                      │
  │                                                      │
  │   5V ────────┬───────────────────── TF-Luna VCC     │
  │              │                                       │
  │   GND ───────┼───────────────┬───── TF-Luna GND     │
  │              │               │                       │
  │   GPIO16 ────│───────────────│───── TF-Luna TX      │
  │   (RX2)      │               │                       │
  │              │               │                       │
  │   GPIO17 ────│───────────────│───── TF-Luna RX      │
  │   (TX2)      │               │                       │
  │              │               │                       │
  │   GPIO12 ────│──── Servo1 (นิ้วซ้าย)                 │
  │              │                                       │
  │   GPIO13 ────│──── Servo2 (นิ้วขวา)                  │
  │              │                                       │
  │              └───── Servo VCC (5V)                   │
  │                                                      │
  └─────────────────────────────────────────────────────┘
```

**TF-Luna Pins:**
```
  ┌─────────────┐
  │  TF-Luna    │
  │  1=VCC (5V) │
  │  2=RX       │ ← ESP32 GPIO17 (TX2)
  │  3=TX       │ → ESP32 GPIO16 (RX2)  
  │  4=GND      │
  └─────────────┘
```

⚠️ **สำคัญ:**
- TF-Luna ใช้ไฟ **5V** (ไม่ใช่ 3.3V)
- TX ของ sensor → RX ของ ESP32
- RX ของ sensor ← TX ของ ESP32

## 1️⃣ Setup

In [1]:
import cv2
import numpy as np
import time
import serial

ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

print("✓ Setup complete")

✓ Setup complete


---
# 📏 Test 1: LIDAR Basic Reading
---

In [3]:
print("="*60)
print("📏 TEST 1: LIDAR Basic Reading")
print("="*60)

# Connect to ESP32
ser = serial.Serial(ESP32_PORT, ESP32_BAUDRATE, timeout=2)
time.sleep(2)
ser.reset_input_buffer()

print("\n🔄 Reading LIDAR 10 times...")
print("   วางพื้นว่างไว้ด้านล่าง LIDAR\n")

readings = []
for i in range(10):
    ser.write(b'L\n')
    time.sleep(0.1)
    response = ser.readline().decode().strip()
    
    if response.startswith("LIDAR:") and "ERR" not in response:
        dist = int(response.split(":")[1])
        readings.append(dist)
        print(f"  [{i+1}] Distance: {dist} mm ({dist/10:.1f} cm)")
    else:
        print(f"  [{i+1}] Error: {response}")
    time.sleep(0.2)

if readings:
    avg = np.mean(readings)
    std = np.std(readings)
    print(f"\n📊 Average: {avg:.1f} mm  Std: {std:.2f} mm")
    if std < 5:
        print("✅ LIDAR readings are stable!")
    else:
        print("⚠️ LIDAR readings have high variance")

ser.close()
print("\n✓ Test complete")

📏 TEST 1: LIDAR Basic Reading

🔄 Reading LIDAR 10 times...
   วางพื้นว่างไว้ด้านล่าง LIDAR

  [1] Distance: 260 mm (26.0 cm)
  [2] Distance: 260 mm (26.0 cm)
  [3] Distance: 260 mm (26.0 cm)
  [4] Distance: 260 mm (26.0 cm)
  [5] Distance: 260 mm (26.0 cm)
  [6] Distance: 260 mm (26.0 cm)
  [7] Distance: 250 mm (25.0 cm)
  [8] Distance: 260 mm (26.0 cm)
  [9] Distance: 260 mm (26.0 cm)
  [10] Distance: 260 mm (26.0 cm)

📊 Average: 259.0 mm  Std: 3.00 mm
✅ LIDAR readings are stable!

✓ Test complete


---
# 📏 Test 2: LIDAR + Gripper Commands
---

In [4]:
print("="*60)
print("📏 TEST 2: LIDAR + Gripper Commands")
print("="*60)

ser = serial.Serial(ESP32_PORT, ESP32_BAUDRATE, timeout=2)
time.sleep(2)
ser.reset_input_buffer()

print("\nAvailable commands:")
print("  L = Read LIDAR")
print("  O = Open gripper")
print("  C = Close gripper")
print("  G<n> = Gripper angle (e.g. G50)")
print("  ? = Status")
print("  Q = Quit")

while True:
    cmd = input("\nCommand: ").strip().upper()
    if cmd == 'Q':
        break
    
    if cmd:
        ser.reset_input_buffer()
        ser.write((cmd + '\n').encode())
        time.sleep(0.3)
        
        while ser.in_waiting:
            print(f"  → {ser.readline().decode().strip()}")

ser.close()
print("✓ Done")

📏 TEST 2: LIDAR + Gripper Commands

Available commands:
  L = Read LIDAR
  O = Open gripper
  C = Close gripper
  G<n> = Gripper angle (e.g. G50)
  ? = Status
  Q = Quit
  → LIDAR:260
  → OK:OPEN:22
  → OK:OPEN:22
  → ANGLE:22 LIDAR:250mm STR:14814
  → OK:OPEN:22
✓ Done


---
# 📐 Calibration 1: LIDAR_TO_GRIPPER_OFFSET
---

## วิธี Calibrate:
1. ย้าย Robot ให้ gripper เกือบแตะพื้น (Z=-64)
2. อ่านค่า LIDAR (L)
3. ค่าที่อ่านได้ = **LIDAR_TO_GRIPPER_OFFSET**

หรือ

1. วางของความสูงที่รู้ (เช่น กล่องสูง 50mm)
2. ย้าย Robot ให้ gripper แตะของพอดี
3. อ่านค่า LIDAR = LIDAR_TO_GRIPPER_OFFSET

In [5]:
print("="*60)
print("📐 CALIBRATION 1: LIDAR_TO_GRIPPER_OFFSET")
print("="*60)

ser = serial.Serial(ESP32_PORT, ESP32_BAUDRATE, timeout=2)
time.sleep(2)

print("\n🔧 Interactive Calibration:")
print("   Press ENTER to read LIDAR, 'Q' to quit\n")

while True:
    cmd = input("Press ENTER to read LIDAR (Q to quit): ").strip().upper()
    if cmd == 'Q':
        break
    
    # Read LIDAR
    ser.reset_input_buffer()
    readings = []
    for _ in range(5):
        ser.write(b'L\n')
        time.sleep(0.1)
        response = ser.readline().decode().strip()
        if response.startswith("LIDAR:") and "ERR" not in response:
            readings.append(int(response.split(":")[1]))
        time.sleep(0.05)
    
    if readings:
        avg = np.mean(readings)
        print(f"   📏 LIDAR Average: {avg:.1f} mm")
        print(f"   → ถ้า gripper แตะพื้น: LIDAR_TO_GRIPPER_OFFSET = {avg:.0f}")
    else:
        print("   ❌ Error reading LIDAR")

ser.close()

📐 CALIBRATION 1: LIDAR_TO_GRIPPER_OFFSET

🔧 Interactive Calibration:
   Press ENTER to read LIDAR, 'Q' to quit

   📏 LIDAR Average: 20.0 mm
   → ถ้า gripper แตะพื้น: LIDAR_TO_GRIPPER_OFFSET = 20
   📏 LIDAR Average: 20.0 mm
   → ถ้า gripper แตะพื้น: LIDAR_TO_GRIPPER_OFFSET = 20
   📏 LIDAR Average: 20.0 mm
   → ถ้า gripper แตะพื้น: LIDAR_TO_GRIPPER_OFFSET = 20
   📏 LIDAR Average: 20.0 mm
   → ถ้า gripper แตะพื้น: LIDAR_TO_GRIPPER_OFFSET = 20


---
# 📐 Calibration 2: Z_MEASURE
---

## หลักการ:
- Z_MEASURE ต้องสูงพอให้ LIDAR มองเห็นวัตถุ
- TF-Luna มีระยะขั้นต่ำ ~20cm (200mm)

## ค่าแนะนำ:
```
Z_MEASURE = 100 ถึง 150
```

ต้องทดสอบว่า LIDAR อ่านค่าได้ที่ Z นั้น

In [7]:
print("="*60)
print("📐 CALIBRATION 2: Z_MEASURE")
print("="*60)

print("""
TF-Luna มีระยะวัดขั้นต่ำ ~20cm (200mm)

ตารางความสูง (สมมุติ offset = 47mm):

  Z_MEASURE | ห่างจากพื้น | LIDAR ห่างพื้น
  ---------|------------|---------------
    -35    |   29 mm    |  ~76 mm  ❌
      0    |   64 mm    | ~111 mm  ❌
    100    |  164 mm    | ~211 mm  ✅
    150    |  214 mm    | ~261 mm  ✅

แนะนำ: Z_MEASURE = 100 ถึง 150
""")

print("วิธีทดสอบ:")
print("1. ย้าย Robot ไปที่ Z ต่างๆ")
print("2. อ่านค่า LIDAR")
print("3. หาค่า Z ที่ต่ำสุดที่ LIDAR ยังอ่านได้")

📐 CALIBRATION 2: Z_MEASURE

TF-Luna มีระยะวัดขั้นต่ำ ~20cm (200mm)

ตารางความสูง (สมมุติ offset = 47mm):

  Z_MEASURE | ห่างจากพื้น | LIDAR ห่างพื้น
  ---------|------------|---------------
    -35    |   29 mm    |  ~76 mm  ❌
      0    |   64 mm    | ~111 mm  ❌
    100    |  164 mm    | ~211 mm  ✅
    150    |  214 mm    | ~261 mm  ✅

แนะนำ: Z_MEASURE = 100 ถึง 150

วิธีทดสอบ:
1. ย้าย Robot ไปที่ Z ต่างๆ
2. อ่านค่า LIDAR
3. หาค่า Z ที่ต่ำสุดที่ LIDAR ยังอ่านได้


---
# 📐 Calibration 3: PIXELS_PER_MM (Camera Scale)
---

In [ ]:
print("="*60)
print("📐 CALIBRATION 3: PIXELS_PER_MM")
print("="*60)
print("""
วิธี Calibrate:
1. วางไม้บรรทัดบนพื้นที่กล้องมองเห็น
2. จับภาพ แล้วใช้เมาส์คลิก 2 จุดที่ห่างกัน เช่น 100mm
3. คำนวณ PIXELS_PER_MM = จำนวน pixel / ระยะ mm
""")

points = []

def mouse_cb(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(f"  Point {len(points)}: ({x}, {y})")
        if len(points) >= 2:
            p1, p2 = points[-2], points[-1]
            dist_px = np.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)
            print(f"  Distance: {dist_px:.1f} pixels")
            
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Calibrate PPM')
cv2.setMouseCallback('Calibrate PPM', mouse_cb)

print("\n📷 Click 2 points on ruler to measure pixel distance")
print("   Press 'C' to calculate, 'R' to reset, 'Q' to quit\n")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    display = frame.copy()
    
    # Draw points
    for i, pt in enumerate(points):
        cv2.circle(display, pt, 5, (0,0,255), -1)
        cv2.putText(display, str(i+1), (pt[0]+10, pt[1]), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    # Draw line between last 2 points
    if len(points) >= 2:
        cv2.line(display, points[-2], points[-1], (0,255,0), 2)
        dist_px = np.sqrt((points[-1][0]-points[-2][0])**2 + 
                          (points[-1][1]-points[-2][1])**2)
        mid = ((points[-2][0]+points[-1][0])//2, (points[-2][1]+points[-1][1])//2)
        cv2.putText(display, f"{dist_px:.1f}px", mid, 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    
    cv2.putText(display, "Click 2 points | C=Calculate | R=Reset | Q=Quit", 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    
    cv2.imshow('Calibrate PPM', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        points = []
        print("  Reset points")
    elif key == ord('c') and len(points) >= 2:
        dist_px = np.sqrt((points[-1][0]-points[-2][0])**2 + 
                          (points[-1][1]-points[-2][1])**2)
        mm = float(input(f"\n  Enter real distance in mm: "))
        ppm = dist_px / mm
        print(f"\n  ✅ PIXELS_PER_MM = {ppm:.4f}")
        print(f"     (Copy this value to 11_lidar_grasp_v11.ipynb)\n")

cap.release()
cv2.destroyAllWindows()

---
# 📋 Summary: Values to Copy
---

In [ ]:
print("="*60)
print("📋 SUMMARY: Copy these values to 11_lidar_grasp_v11.ipynb")
print("="*60)

print("""
# === Configuration Section (Cell 3) ===

# จาก Calibration 3:
PIXELS_PER_MM = 2.7703  # ปรับตามค่าที่ได้

# จาก Calibration 2:
Z_MEASURE = 100  # ปรับตามค่าที่ได้ (ต้องให้ LIDAR อ่านได้)

# จาก Calibration 1:
LIDAR_TO_GRIPPER_OFFSET = 47  # ปรับตามค่าที่ได้
""")